In [ ]:
import matplotlib.pyplot as plt
import polars as pl
import requests_cache

from moonbox import get_oneday, parse_oneday, draw_moon

In [ ]:
session = requests_cache.CachedSession("cache")

year = 2025

dates = (
    pl.select(pl.date_range(pl.date(year, 1, 1), pl.date(year, 12, 31)))
    .to_series()
    .to_list()
)

data = [
    {"date": date} | parse_oneday(get_oneday(date.isoformat(), session=session))
    for date in dates
]

In [ ]:
def chunk(lst, pred, min_len=2):
    out = []
    chunk = None
    for elt in lst:
        if chunk is None:
            if pred(elt):
                chunk = [elt]
            else:
                pass
        elif pred(elt) and len(chunk) + 1 > min_len:
            out.append(chunk)
            chunk = [elt]
        else:
            chunk.append(elt)

    return out


# group into lunar months
lunar_months = chunk(data, lambda x: x["phase"] == "New Moon")

n_months = len(lunar_months)
longest_month = max([len(x) for x in lunar_months])
assert longest_month == 30

In [ ]:
plt.ioff()

# set up the calendar geometry
canvas_size = (28.0, 22.0)
calendar_width_prop = 0.95
spacer_x_prop = 0.1
spacer_y_x_ratio = 8.0
calendar_middle = 0.4  # middle of calendar is this fraction up the canvas

date_font_size = 20
title_font_size = 60
signature_font_size = 30

# derived positions
calendar_width = canvas_size[0] * calendar_width_prop
moon_radius = calendar_width / (2 * longest_month + spacer_x_prop * (longest_month - 1))
spacer_x = spacer_x_prop * moon_radius
spacer_y = spacer_y_x_ratio * spacer_x

calendar_height = n_months * 2 * moon_radius + (n_months - 1) * spacer_y

calendar_x = (canvas_size[0] - calendar_width) / 2
calendar_y = (canvas_size[1] - calendar_height) * calendar_middle

fig = plt.figure(figsize=canvas_size)
ax = fig.add_axes([0, 0, 1, 1])
ax.set_xlim(0, canvas_size[0])
ax.set_ylim(0, canvas_size[1])

ax.set_facecolor("black")

solar_month = None

for row, month in enumerate(lunar_months):
    for col, datum in enumerate(month):
        if datum["date"].month == 1 and datum["date"].day == 1:
            label = datum["date"].strftime("%b %-d, %y")
            solar_month = datum["date"].month
            date_align = "left"
        elif col == 0 or datum["date"].month != solar_month:
            label = datum["date"].strftime("%b %-d")
            solar_month = datum["date"].month
            date_align = "left"
        else:
            label = datum["date"].strftime("%-d")
            date_align = "center"

        f = datum["illumination"] / 100
        direction = datum["phase"].split()[0].lower()

        moon_x = calendar_x + (2 * moon_radius + spacer_x) * col + moon_radius
        moon_y = calendar_y + (2 * moon_radius + spacer_y) * row + moon_radius

        draw_moon(
            ax,
            x=moon_x,
            y=moon_y,
            radius=moon_radius,
            f=f,
            direction=direction,
            dark="0.2",
        )
        # date labels
        if date_align == "left":
            date_x = moon_x - moon_radius
        elif date_align == "center":
            date_x = moon_x
        else:
            raise RuntimeError

        ax.text(
            x=date_x,
            y=moon_y + moon_radius,
            s=label,
            horizontalalignment=date_align,
            verticalalignment="bottom",
            color="white",
            fontsize=date_font_size,
        )

# title
ax.text(
    x=15.0,
    y=20.0,
    s=f"{year} Lunar Months",
    verticalalignment="top",
    horizontalalignment="center",
    fontsize=title_font_size,
    color="white",
)

# signature
ax.text(
    x=canvas_size[0],
    y=0.0,
    s="Scott Olesen",
    verticalalignment="bottom",
    horizontalalignment="right",
    fontsize=signature_font_size,
    color="white",
)

# do not show the figure; rendering in VSCode is very slow
fig.savefig("../output/tmp.png", dpi=72)

In [ ]:
# production quality
fig.savefig("../output/lunar_calendar.png", dpi=300)
fig.savefig("../output/lunar_calendar.pdf")